In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import theano
import theano.tensor as T
import time

import lasagne as L

import sys
sys.path.insert(0, '../SimpleRNNLM/')

%aimport VHRED
from mt_load import load_mt, get_mt_voc, get_w2v_embs
from load_subtle import load_subtle
from diverse_beam_search import diverse_beam_search

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 780 (CNMeM is enabled with initial size: 30.0% of memory, cuDNN 4007)


In [2]:
# remember, now the pad value is the same as the <utt_end> token

pad_value = -1 # <utt_end>'s vector is the last one

In [ ]:
t0 = time.time()
subtle_path = "/pio/data/data/mtriples/"

train_subtle = load_subtle(subtle_path, split=True, trim=200)
print time.time() - t0

In [3]:
mt_path = "/pio/data/data/mtriples/"
# mt_path = "../DATA/MovieTriples_Dataset/"

train, valid, test = load_mt(path=mt_path, split=True, trim=200)
idx_to_w, w_to_idx, voc_size, freqs = get_mt_voc(path=mt_path, train_len=len(train))

In [4]:
word2vec_embs, word2vec_embs_mask = get_w2v_embs(path=mt_path)

w2v_train_mask = np.where(word2vec_embs_mask[:,0] == 1)[0]

In [21]:
%%time
vhred_net = VHRED.VHRED(voc_size=voc_size,
                        emb_size=300,
                        lv1_rec_size=300, 
                        lv2_rec_size=300, 
                        out_emb_size=300,
                        latent_size=20,
                        num_sampled=200,
                        ssoft_probs=freqs,
                        train_emb=False,
                        kl_annealing=True)

Building the model...
Compiling theano functions...
Building a network for generating...
Done
CPU times: user 51 s, sys: 787 ms, total: 51.8 s
Wall time: 51.6 s


In [22]:
# with KL annealing, some information is put into prior and posterior
# avg KL div per word on valid: 0.049
vhred_net.load_params('trained_models/vhred_hredEmbFixed_300_300_300_300_20_Anneal2M_drop25_ssoft200unigr_bs30_cut200_early5.npz')

In [18]:
# this has no KL annealing, the prior and posterior are very close, because there is no information coded there
# avg KL div per word on valid: 7e-6
vhred_net.load_params('trained_models/vhred_hredEmbFixed_300_300_300_300_10_noAnneal_drop25_ssoft200unigr_bs30_cut200_early5.npz')

In [24]:
vhred_net.validate(valid[:30], 30)

0.04151616809414882

In [44]:
vhred_net.load_params_from_HRED('trained_models/subtleFixed_300_300_300_300_ssoft200unigr_bs30_cut200_early5.npz')

In [82]:
vhred_net.train_one_epoch(train[3000:6000], 30)

Done 10 batches in 1.15s	training loss:	5.675020
0.00654025608674
Done 20 batches in 2.75s	training loss:	5.616165
0.00714027928188
Done 30 batches in 4.19s	training loss:	5.584745
0.00774030247703
Done 40 batches in 5.70s	training loss:	5.560396
0.00834032613784
Done 50 batches in 7.03s	training loss:	5.550236
0.00894034933299
Done 60 batches in 8.68s	training loss:	5.530466
0.00954037252814
Done 70 batches in 9.84s	training loss:	5.516066
0.0101403957233
Done 80 batches in 11.20s	training loss:	5.507935
0.0107404189184
Done 90 batches in 12.65s	training loss:	5.498226
0.0113404421136
Done 100 batches in 14.03s	training loss:	5.490364
0.0119404653087


5.49036352239

In [84]:
L.layers.get_all_param_values(vhred_net.train_net)[-1]

array(0.012000467628240585, dtype=float32)

In [36]:
vhred_net.save_params('trained_models/vhred_ep1_dropAll')

In [32]:
mu0 = np.zeros((1,10))
mu1 = np.zeros((1,10))
sig0 = np.array([[.1,.1,.2,.02,.5,.006,.7,.0009,.004,.1]])
sig1 = np.array([[.1,.1,.2,.02,.5,.006,.7,.001,.004,.1]])
        
res = (sig0 / sig1).sum(axis=1) + ((mu1 - mu0) / sig1 * (mu1 - mu0)).sum(axis=1) - \
      mu0.shape[1] + np.log(sig1.prod(axis=1) / sig0.prod(axis=1))
res *= .5
res

array([ 0.00268026])

In [33]:
def print_utt(utt):
    return ' '.join([idx_to_w[x] for x in utt])

def rnd_next_word(probs, size=1):
    return np.random.choice(np.append(np.arange(probs.shape[0]-1), -1).astype(np.int32), 
                            size=size, p=probs)

def utt_to_array(utt):
    arr = np.array([w_to_idx.get(w, w_to_idx['<unk>']) for w in utt])[np.newaxis].astype(np.int32)
    arr[arr == -voc_size] = -1
    return arr

def context_summary(context, lookup=True):
    con_init = np.zeros((1, vhred_net.lv2_rec_size), dtype=np.float32)
    for utt in context:
        con_init, z = vhred_net.get_new_con_init_fn(utt_to_array(utt) if lookup else utt, con_init)
    return con_init, z

In [27]:
context = map(lambda x: np.array(x, dtype=np.int32)[np.newaxis], train[99:101])
lookup = False
for u in context:
    print print_utt(u[0])

<s> i couldn ' t say . </s>
<s> you were a prosecutor . </s>


In [8]:
context = ['<s> hi . </s>'.split(), '<s> hello , what \' s up ? </s>'.split()]
lookup = True

In [14]:
context = ['<s> yeah , okay . </s>'.split(), '<s> well , i guess i \' ll be going now . </s>'.split()]
lookup = True

In [17]:
context= ['<s> what would the table think about if it could think ? </s>'.split()]
lookup = True

In [19]:
context= ['<s> i saw a pretty good movie yesterday . </s>'.split()]
lookup = True

In [31]:
context= ['<s> hi . </s>'.split()]
lookup = True

In [34]:
beam_size = 20
group_size = 2
con_init, z = context_summary(context, lookup)
W = L.layers.get_all_param_values(vhred_net.train_net)[40]
b = L.layers.get_all_param_values(vhred_net.train_net)[41]
dec_init = np.repeat(np.tanh(np.hstack([con_init, z]).dot(W) + b), beam_size, axis=0)

mean = True

beamsearch = diverse_beam_search(beam_size, group_size, dec_init, voc_size, vhred_net, 
                                 init_seq=utt_to_array('<s> '.split()), rank_penalty=0, group_diversity_penalty=1, 
                                 seq_diversity_penalty=1, verbose_log=False)

# print print_utt(beamsearch)

len_bonus = lambda size: 0#np.log(size)**2

def fn_score(x, y, mean=mean, len_bonus=len_bonus):
    denom = (x.size - 1) if mean else 1
    return (y + len_bonus(x.size)) / denom

sort1 = sorted(beamsearch, key=lambda (x,y): fn_score(x, y), reverse=True)
sort2 = sorted(beamsearch, key=lambda x: ' '.join(print_utt(x[0][1:-1])))

for utt, scr in sort1:
    print '{:.3f}  '.format(fn_score(utt, scr)), print_utt(utt[1:-1])
    print ''

-0.692   yes .

-0.921   no .

-1.319   no . i ' d like to ask you something .

-1.319   no . i ' m not .

-1.321   no . i ' d like to see you .

-1.359   yes .

-1.359   yes .

-1.359   yes .

-1.359   yes .

-1.359   yes .

-1.359   yes .

-1.359   yes .

-1.359   yes .

-1.359   yes .

-1.377   yes , sir .

-1.411   yes , but i ' m not .

-1.545   yes , but i ' m not sure .

-1.587   no .

-1.587   no .

-1.587   no .

-1.587   no .

-1.587   no .

-1.587   no .

-1.587   no .

-1.587   no .

-1.628   no . i ' d like to ask you something . i was thinking about it .

-1.636   no . i ' d like to ask you something . i was thinking about it , but i didn ' t know what happened to me .

-1.655   no . i ' d like to ask you something . i was thinking about it , but i didn ' t know what happened to him .

-1.659   no . i didn ' t know what happened to me .

-1.666   no . i ' d like to ask you something . i was thinking about it , but i didn ' t know what happened .

-1.706   no . i didn ' t 

In [35]:
beam_size = 20
group_size = 2

for i in xrange(0, 100, 3):
    context = map(lambda x: np.array(x, dtype=np.int32)[np.newaxis], test[i:i+2])
    lookup = False
    for u in context:
        print print_utt(u[0])
        
    con_init, z = context_summary(context, lookup)
    W = L.layers.get_all_param_values(vhred_net.train_net)[40]
    b = L.layers.get_all_param_values(vhred_net.train_net)[41]
    dec_init = np.repeat(np.tanh(np.hstack([con_init, z]).dot(W) + b), beam_size, axis=0)

    beamsearch = diverse_beam_search(beam_size, group_size, dec_init, voc_size, vhred_net,
                                     init_seq=utt_to_array('<s> '.split()), )
    
    len_bonus = lambda size: 0#np.log(size)**2

    mean = True
        
    def fn_score(x, y, mean=mean, len_bonus=len_bonus):
        denom = (x.size - 1) if mean else 1
        return (y + len_bonus(x.size)) / denom

    for utt, scr in sorted(beamsearch, key=lambda (x,y): fn_score(x, y), reverse=True)[:1]:
        print '{:.3f}  '.format(fn_score(utt, scr)), print_utt(utt[1:-1])
        print ''
        
# for utt in beamsearch:
#     print_utt(utt)
#     print ''

<s> you ' re not going to let me die , are you ! ? <person> ' t let me die ! ignoring the <unk> </s>
<s> when we start running suck up your breath . you ' re going to get the wind knocked out of you . </s>
-1.042   what ' s that ?

<s> <person> says you have to be like me . </s>
<s> he does . </s>
-0.811   what do you mean ?

<s> <person> . what ' s your <number> ? </s>
<s> reservation . </s>
-1.468   right .

<s> what the hell ' s going on here ? </s>
<s> we ' re in love with your roommate . </s>
-1.591   in the way .

<s> i ' m going to the market . i ' ll be back in a bit . </s>
<s> can i go with you ? </s>
-0.709   sure .

<s> yes - </s>
<s> <person> ! <person> these men ! escort them to the <unk> ! </s>
-0.974   yes , sir .

<s> sure . </s>
<s> you can let me down now . </s>
-1.057   you ' re not going anywhere .

<s> i don ' t know . </s>
<s> we got a <unk> , you know . <person> -- did you hear what i said . ? </s>
-0.793   i don ' t know .

<s> what ' s this for ? </s>
<s> for <

In [7]:
hred_net.validate(test, 30)

Done 100 batches in 4.07s
Done 200 batches in 8.39s
Done 300 batches in 12.95s
Done 400 batches in 17.68s
Done 500 batches in 21.70s
Done 600 batches in 26.34s
Done 700 batches in 31.03s
Done 800 batches in 35.56s
Done 900 batches in 40.01s
Done 1000 batches in 44.49s
Done 1100 batches in 48.81s
Done 1200 batches in 53.21s
Done 1300 batches in 57.49s
Done 1400 batches in 61.62s
Done 1500 batches in 66.13s
Done 1600 batches in 70.65s
Done 1700 batches in 74.90s
Done 1800 batches in 79.24s
Done 1900 batches in 83.80s
Done 2000 batches in 88.35s
Done 2100 batches in 92.88s
Done 2200 batches in 97.45s
Done 2300 batches in 101.66s
Done 2400 batches in 106.09s


3.2819798801888118

In [8]:
'''full softmax, bs=30'''
# train, 1 dir, 1 epoch: 3.485554076321884
# val: 3.455356876018342

# train, 2 dir, concat, 1 epoch: 3.4864403798772239
# val: 3.4579001751897063

# train, 2 dir, L2 + concat, 1 epoch: 3.4881669768474675
# val: 3.4584704095551695
# training time: ~4700s

'''sampled softmax'''
# bs=30
# train, 2 dir, L2 + concat, 1 epoch: 3.486180601246621
# val: 3.4811877499289308
# training time: ~2300s

# bs=60
# train, 2 dir, L2 + concat, 1 epoch: 3.5235153449672456
# val: 3.5063306987542759
# training time: ~1900s